In [1]:
#Las librerias clasicas
import numpy as np
import pandas as pd

# Estas son las que corresponden a dash y plotly 
import dash 
from dash import dcc
from dash import html
from dash.dependencies import Input, Output, State
from dash import dash_table
import dash_bootstrap_components as dbc

import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio

# Cargamos archivos

In [2]:
nat_rmsd = [np.genfromtxt("assets/rmsd_"+i+".dat") for i in ["nat_ncst","nat_psn1","nat_aph1","nat_pen2","toda_nat"]]
mut_rmsd = [np.genfromtxt("assets/rmsd_"+i+".dat") for i in ["mut_ncst","mut_psn1","mut_aph1","mut_pen2","toda_mut"]]
mut1_gamma = [np.genfromtxt("assets/rmsd_"+i+".dat") for i in ["toda_mut","toda_mut-mut","toda_mut-apo"]]
mut_ncst = [np.genfromtxt("assets/rmsd_"+i+".dat") for i in ["mut_ncst","mut_ncst-mut","mut_ncst-apo"]]
mut_psen = [np.genfromtxt("assets/rmsd_"+i+".dat") for i in ["mut_psn1","mut_psn1-mut","mut_psn1-apo"]]
mut_aph1 = [np.genfromtxt("assets/rmsd_"+i+".dat") for i in ["mut_aph1","mut_aph1-mut","mut_aph1-apo"]]
mutpen2 = [np.genfromtxt("assets/rmsd_"+i+".dat") for i in ["mut_pen2","mut_pen2-mut","mut_pen2-apo"]]


df  = pd.read_csv('assets/numerado.csv')
aa = df[df["Sistema"]=="Estructura Nativa"]["Subunidad"].values+": "+df[df["Sistema"]=="Estructura Nativa"]["Aminoacido"].values+"-"+df[df["Sistema"]=="Estructura Nativa"]["Corregida"].values.astype("str")


nat_rmsf = np.genfromtxt('assets/rmsf_nat.dat')
mut_rmsf = np.genfromtxt("assets/rmsf_mut_apo.dat")
mut_rmsf2 = pd.read_csv("assets2/rmsf_mut2_apo.dat",sep="\s+")


#Ubicaciones especificas
asp = pd.read_csv("assets/mut_dist_6ct_asp.dat",sep="\s+")
p31 = pd.read_csv("assets/mut_dist_6ct_p31.dat",sep="\s+")
popc = pd.read_csv("assets/mut_dist_6ct_popc.dat",sep="\s+")
tm2  = pd.read_csv("assets/mut_dist_6ct_tm2-3.dat",sep="\s+")

asp1 = pd.read_csv("assets/apo_dist_6ct_asp.dat",sep="\s+")
p311 = pd.read_csv("assets/apo_dist_6ct_p31.dat",sep="\s+")
popc1 = pd.read_csv("assets/apo_dist_6ct_popc.dat",sep="\s+")
tm21  = pd.read_csv("assets/apo_dist_6ct_tm2-3.dat",sep="\s+")

difz31 = np.loadtxt("assets/apo_difz_p31_6ct.csv", delimiter=",")
difzpopc = np.loadtxt("assets/apo_difz_centromem_6ct.csv", delimiter=",")
difz311 = np.loadtxt("assets/mut_difz_p31_6ct.csv", delimiter=",")
difzpopc1 = np.loadtxt("assets/mut_difz_centromem_6ct.csv", delimiter=",")

#distancias asparticos
nat_dist_d = np.genfromtxt('assets/asp_dist_nat.dat')
mut_dist_d = np.genfromtxt('assets/asp_dist_mut.dat')

#Contactos
apo_cont = np.loadtxt("assets/contactos_apo.csv", delimiter=",")
mut_cont = np.loadtxt("assets/contactos_l166p.csv",delimiter=",")
helix_df  = pd.read_csv("assets/helix_contactos.csv")

In [3]:
def anchomem(dd):
    extremos = dd[:,0][(dd[:,0] >= -21.5) & (dd[:,0] <= 21.5)][0], dd[:,0][(dd[:,0] >= -21.5) & (dd[:,0] <= 21.5)][-1]
    medios = dd[:,0][(dd[:,0] >= -18.5) & (dd[:,0] <= 18.5)][0], dd[:,0][(dd[:,0] >= -18.5) & (dd[:,0] <= 18.5)][-1]

    extremos = np.where(dd[:,0]==extremos[0])[0][0], np.where(dd[:,0]==extremos[1])[0][0]
    medios = np.where(dd[:,0]==medios[0])[0][0], np.where(dd[:,0]==medios[1])[0][0]

    res1 = dd[:,7][extremos[0]:medios[0]]-dd[:,9][extremos[0]:medios[0]]
    sumaleid1  = np.where(res1 == res1[min(range(len(res1)), key = lambda i: abs(res1[i]-0))])[0][0]
    inter1 = np.abs(dd[:,0][extremos[0]+sumaleid1])

    res2 = dd[:,7][medios[1]:extremos[1]]-dd[:,9][medios[1]:extremos[1]]
    sumaleid2  = np.where(res2 == res2[min(range(len(res2)), key = lambda i: abs(res2[i]-0))])[0][0]
    inter2 = np.abs(dd[:,0][medios[1]+sumaleid2])
    mem_ancho = inter1+inter2
    return str(mem_ancho)

# Esqueleto del dash
## Barside
[Para ver cositas del sidebar](https://stackoverflow.com/questions/62732631/how-to-collapsed-sidebar-in-dash-plotly-dash-bootstrap-components)

In [4]:
# F O R M A T O   D E   L A   B A R R A  Y  C O N T E N I D O

# styling the sidebar
SIDEBAR_STYLE = {
    "position": "fixed",  #Se mantiene aunque tu hagas mas chica la ventana
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "22rem",     #Ancho del sidebar 
    "padding": "2rem 1rem", #Cuanto de espacio hay arriba antes de que empiece el texto y el espacio que debe quedar a la derecha
    "background-color": "#f8f9fa",
}

SIDEBAR_HIDEN = {
    "position": "fixed",
    "top": 62.5,
    "left": "-16rem",
    "bottom": 0,
    "width": "16rem",
    "height": "100%",
    "z-index": 1,
    "overflow-x": "hidden",
    "transition": "all 0.5s",
    "padding": "0rem 0rem",
    "background-color": "#f8f9fa",
}

# padding for the page content
CONTENT_STYLE = {
    "margin-left": "25rem",  #Cuanto deja de espacio dsde TODA la pantalla
    "margin-right": "0rem",  #Margen a la dercha
    "padding": "0rem 0rem",  #Espacio arriba y a la derecha
}

CONTENT_STYLE1 = {
    "transition": "margin-left .5s",
    "margin-left": "2rem",
    "margin-right": "2rem",
    "padding": "2rem 1rem",
    "background-color": "#f8f9fa",
}
sidebar = html.Div(
    [
        dcc.Markdown("""**Selecciona el el analisis que quieres ver**"""),
        html.Hr(),
        dbc.Nav(
            [
                dbc.NavLink("Introducción", href="/", active="exact"),
                dbc.NavLink("RMSD", href="/rmsd", active="exact"),
                dbc.NavLink("RMSF", href="/rmsf", active="exact"),
                dbc.NavLink("Analisis de Estructura Secundaria", href="/sec-estruc", active="exact"),
                dbc.NavLink("HelAnal", href="/helanal", active="exact"),
                dbc.NavLink("Contactos membranales", href="/contact", active="exact"),
                dbc.NavLink("Ubicación de regiones", href="/ubicacion", active="exact"),
                dbc.NavLink("Componentes principales", href="/pca", active="exact"),
                dbc.NavLink("Accesibilidad al solvente", href="/h2o", active="exact"),
                dbc.NavLink("Asparticos cataliticos", href="/asp-cat", active="exact"),
                dbc.NavLink("Integridad membrana", href="/lipyphilic", active="exact"),
            ],
            vertical=True,
            pills=True,
        ),
    ],
    id="sidebar",
    style=SIDEBAR_STYLE,
)

content = html.Div(id="page-content", children=[], style=CONTENT_STYLE)


## Encabezado

In [5]:
encabezado  = html.Div([
    dbc.Row(dbc.Button("Sidebar", outline=True, color="secondary", className="mr-1", id="btn_sidebar")),
    dbc.Row([
    
    dbc.Col(html.A(html.Img(src="assets/UNAM.png",style={'height':'130%', 'width':'95%'},className="d-flex align-self-center"),
                   href="https://www.unam.mx/",target='_blank'), 
            width={"size":2,"offset":0}),

    dbc.Col(html.H2("Análisis y-Secretasa",style={"color":"#8a070d"},className="text-center" "mb-4" "h50"),
            width={"size":4,"offset":2}),

    dbc.Col(html.A(html.Img(src="assets/mdcbq.svg",style={'height':'150%', 'width':'110%'},className="d-flex align-self-center"),
                   href="https://www.mdcbq.posgrado.unam.mx/home",target='_blank'),
            width={"size":2,"offset":1})
],justify="center", align="center")])

## Introducción y RMSD

In [6]:
Intro = html.Div([
    html.H4("Introducción"),
    html.P([html.Span("Esta página fue diseñada mediante dash-plotly y otras librerias de python, tiene como objetivo ayudar a explorar los resultados obtenidos por dinámica molecuar para el estudio de y-Secretasa, de una manera interactiva. En esta seccion encontraras una breve descripción de los objetos con los cuales podras interactuar.")]),
    html.Span("A lo largo de las paginas las instrucciónes se marcan en azul",style={"color":"#052080"}),
    html.H4("Objetos interactivos"),
    html.H6("Gráficos"),
    html.P("Diseñamos los gráficos para analizar los datos de una forma rápida y fácil. Ten en cuenta que si colocas el cursor sobre un punto, se mostrará la información asociada, y si vas a la esquina superior derecha del gráfico, encontrarás una barra de herramientas, que te permitira desplazarte dentro de la figura o descargar la imagen. Los graficos se pueden descargar como figuras estaticas. En el siguiente menu desplegable podras escoger el tipo de formato en el cual se descargaran las imagenes (de manera predeterminada esta png). "),
    html.Br(),
    dbc.Row([
        dbc.Col(html.Span("Escoge el formato en cual quieres descargar tus plots",style={"color":"#052080"})),
        dbc.Col(dcc.Dropdown(
            id="format-dw-img",
            options=[{'label': 'svg', 'value': 'svg'},
                     {'label': 'png', 'value': 'png'},
                     {'label': 'jpeg', 'value': 'jpeg'},
                     {'label': 'webp', 'value': 'webp'}],
            value='png',style={'width':"45%"}))],
        justify="center", align="center"),
])

p_container = html.Div([
    html.H4("Introducción"),
    html.P("La desviación cuadrática media o RMSD por sus siglas en inglés es una medida de uso común cuando el objetivo se centra en comparar la distancia o similitud conformacional de dos estructuras; o de otro modo analizar cambios conformacionales de una sola estructura a lo largo del tiempo. Para calcular el RMSD es necesario realizar dos pasos importantes 1) Paso de alineación: en donde se establece una relación uno a uno de los átomos equivalentes de ambas estructuras 2) Paso de superposición óptima: en donde se rota y traslada la estructura de manera que se minimice la suma ponderada de los cuadrados de las distancias de cada uno de los átomos56. En casos de dinámica molecular, en donde se requiere analizar la similitud conformacional a lo largo del tiempo, se puede expresar el cálculo del RMSD como:"),
    dbc.Row([dcc.Markdown('$RMSD(x,x^{ref}) =\\sqrt{\\frac{1}{N} \\sum_{i=1}^{N}[(R \\cdot x_{i}(t)+τ)-x_{i}^{ref}]^{2}}$',mathjax=True)],justify="center",),
    html.P("Que básicamente representa la suma de las distancias euclidianas de cada uno de los átomos, entre una estructura xref y otra estructura x cuando se le aplica un factor de Rotación R y traslación τ que minimizan estas distancias."),
    html.P("Generalmente, los cálculos de RMSD se realizan utilizando únicamente el esqueleto de la proteína, ya que los cambios conformacionales de las cadenas laterales de cada uno de los residuos no son representativos de un cambio conformacional general."),
    html.Br(),
    dbc.Row([
        dbc.Col([html.Span("Observemos los graficos por:  ",style={'display':'block'}),
                 dcc.Dropdown(
                    id="tipormsd",
                    options=[{'label': 'Por sistema', 'value': 'sis'},
                             {'label': 'Por subunidad', 'value': 'comp'}],
                    value='comp',style={'width':"70%",'display':'block'})]),
        dbc.Col([html.Span("Sistema: ",style={'display':'block'}),
                 dcc.Dropdown(
                    id="estruc",
                    options=[{'label': 'Estructura Nativa', 'value': 'nat'},
                             {'label': 'Estructura Nativa l166p', 'value': 'mut'}],
                    value='nat',style={'width':"80%",'display':'block'})]),
        dbc.Col([html.Span("Subunidad: ",style={'display':'block'}),
                 dcc.Dropdown(
                    id="sub1",
                    options=[{'label': 'NCST', 'value': 'ncst'},
                             {'label': 'PSN1', 'value': 'psn1'},
                             {'label': 'APH1', 'value': 'aph1'},
                             {'label': 'PEN2', 'value': 'pen2'},
                             {'label': 'Toda', 'value': 'all'}],
                    value='psn1',style={'width':"55%",'display':'block'})])]),
    
    html.Div(id="plots-rmsd"),
])

## RMSF y Estructura secundaria

In [7]:
rmsf = html.Div([
    html.H4("Introducción"),
    html.P("La fluctuación cuadrática media (RMSF) representa una forma de cuantificar la flexibilidad de una molécula o una región de esta. A diferencia del RMSD que representa el desplazamiento promedio de la estructura en un instante de la simulación, el RMSF denota la fluctuación o vibración de un átomo o conjunto de átomos, con relación a una estructura de referencia"),
    html.P([html.Span("En el caso del cálculo de RMSF, para una molécula a lo largo de una simulación, se cuenta con un conjunto de estructuras que representan la evolución temporal de la molécula. La estructura de referencia usada puede ser la "),
            html.B("estructura media"),
            html.Span("que es aquella en donde cada átomo se encuentra en la posición promedio del conjunto de posiciones, o la "),
            html.B("estructura representativa"),
            html.Span("que es la que tiene más similitudes con cada estructura del conjunto, el grado de similitud se realiza mediante el computo de RMSD por pares para cada una de las integrantes del conjunto")]),
    html.P("La operación utilizada para obtener el RMSF de un solo átomo i, es la siguiente:"),
    dbc.Row([dcc.Markdown('$RMSF_{i} =\\sqrt{\\frac{1}{T} \\sum_{j=i}^{T}(\\vec{r_{j,i}}-\\frac{1}{T}\\sum_{k=i}^{T}\\vec{r_{k,l}})^2}$',mathjax=True)],justify="center",),
    html.P("Que representa la distancia promedio entre el átomo i de la estructura T y el átomo i de la estructura media/representativa, en donde se asume que las coordenadas r fueron rotadas y trasladadas para minimizar el RMSD"),

    dbc.Row([
        html.Span("Observemos como se ve cada estructura ",style={'display':'block'})]),
    html.Br(),
    html.H6("Númeración para las subunidades proteicas"),
    html.P("Puedes escoger ver los analisis que involucren información por residuo  Con 1)Numeración salida de la simulación de AMBER lo cual da facilidad para ubicar los residuos, 2)Númeracion corregida para cada subunidad proteica y mostrando numeración basada en AB para el sustrato o 3)Numeración corregida para cada subunidad proteica y numeración basada en C83 como sustrato"),
    html.Br(),
    dbc.Row([
    dbc.Col(html.Span("Escoge el formato en cual quieres numerar",style={"color":"#052080"})),
        dbc.Col(dcc.Dropdown(
            id="format-dw-pot",
            options=[{'label': 'Simulación AMBER', 'value': 'amber'},
                     {'label': 'Por proteina (Num AB)', 'value': 'ab'},
                     {'label': 'Por proteina (Num C83)', 'value': 'c83'}],
            value='ab',style={'width':"75%"}))],
        justify="center", align="center"),
    html.Br(),
    html.Div(id="plots-rmsf"),
    html.Br(),
    html.B("Cambios que estan en PS1"),
    html.P("Es mas rigida la mutada en el loop TM2-TM3"),
    html.P("Se hace mas rigido el extremo TM6C y TM7N"),
    html.P("Se hace mas rigido TM9, el extremo mas pegado a APH1"),
    html.B("Cambios que estan en APH1"),
    html.P("Se hace mas rigido Loop TM2-TM3")
])

ss = html.Div([
    html.H4("Analisis de estructura secundaria"),
    html.P("Veamos como se comporta la estructura de cada uno de los residuos de aminoácidos, los computos fueron relidazdos mediante CPPTRAJ con la función secstruct"),
    dbc.Row([
        html.Span("Observemos como se ve la estructura secundaria del sistema:   ",style={'display':'block'}),
        dcc.Dropdown(
                id="ss-sub",
                options=[{'label': 'Estructura Nativa', 'value': 'nat'},
                         {'label': 'Estructura Nativa L166P', 'value': 'mut'}],
                value='nat',style={'width':"70%",'display':'block'})]),
    html.Br(),
    html.H6("Númeración para las subunidades proteicas"),
    html.P("Puedes escoger ver los analisis que involucren información por residuo  Con 1)Numeración salida de la simulación de AMBER lo cual da facilidad para ubicar los residuos, 2)Númeracion corregida para cada subunidad proteica y mostrando numeración basada en AB para el sustrato o 3)Numeración corregida para cada subunidad proteica y numeración basada en C83 como sustrato"),
    html.Br(),
    dbc.Row([
    dbc.Col(html.Span("Escoge el formato en cual quieres numerar",style={"color":"#052080"})),
        dbc.Col(dcc.Dropdown(
            id="format-dw-pot",
            options=[{'label': 'Simulación AMBER', 'value': 'amber'},
                     {'label': 'Por proteina (Num AB)', 'value': 'ab'},
                     {'label': 'Por proteina (Num C83)', 'value': 'c83'}],
            value='ab',style={'width':"75%"}))],
        justify="center", align="center"),
    html.Br(),
    html.H4("Cambios importantes Primera replica"),
    html.P([html.B("Se rompe helicidad en TM3 por la prolina entre PS 159-162")]),
    html.P("Se debilita la interaccion beta que habia entre TM4 y el extremo de Cterminal PEN2"),
    html.P("Las laminas beta pierden residuos, se vuelven betas bridge aislados en donde solo estan 287 y 382"),
    html.P("Se estructura como helice 3-10 el loop de APH1 TM6-TM7"),
    
    html.H4("Cambios importantes Segunda replica"),
    html.P("Se rompe estructura  de NCST en la parte intracelular"),
    html.P([html.B("Se rompe helicidad en TM3")]),
    html.P("Se desestructura yuxtamembranal de APH1 TM7"),
    html.Div(id="ss-div"),
    html.Br(),
    
])

## Helanal y Contactos membranales

In [8]:
tilt =  html.Div([
    html.H4("Análisis de las hélices"),
    html.Br(),
    html.P("HELANAL cuantifica la geometría de las helices en proteínas en función de sus átomos Cα. Puede determinar características estructurales locales como la torsión local y la elevación, el ángulo torsional virtual, los orígenes de la hélice local y los ángulos de flexión entre ejes de hélice locales sucesivos."),
    html.Br(),
    html.P("En este apartado se realizan analisis de cada una de las 19 helices membranales de γ-secretasa"),
    html.Br(),
    dbc.Col([
        html.Span("Selecciona el analisis de relacionado a las hélices que te gustaria ver :  ",style={'display':'block'}),]),
    dbc.Col([
        dcc.Dropdown(
                id="helanalanal",
                options=[{'label': 'Tilt/Inclinacion', 'value': 'tilt'},
                        {'label': 'Ángulos entre helices', 'value': '2he'},
                        {'label': 'Kink/torcedura', 'value': 'kinky'}],
                value='tilt',style={'width':"70%",'display':'block'})]),
        
    html.Div(id="analplot")
                 ])

contete = html.Div([
    html.H4("Contactos Membranales"),
    html.Br(),
    html.P("Se realizaron todos los contactos de los residuos membranales de GS contra sigo mismos. Los contactos se presentan en mapas de calor en donde el valor de z representa el contacto normalizado por número de contactos posibles y por tiempo. Se considera contacto aquello que este a 5 Å de distancia."),
    html.Br(),
    html.P("Selecciona por favor que sistemas quieres comparar."),
    html.Br(),
    dbc.Row([       
        dcc.Dropdown(
            id="contact1",
            options=[{'label': 'Estructura Apo', 'value': 'Apo'},
                     {'label': 'Estructura Apo l166p', 'value': 'Mut'}],
            value='Apo',style={'width':"45%",'display':'block'}),

        dcc.Dropdown(
            id="contact2",
            options=[{'label': 'Estructura Apo', 'value': 'Apo'},
                     {'label': 'Estructura Apo l166', 'value': 'Mut'}],
            value='Mut',style={'width':"45%",'display':'block'})]),
     html.Div(id="plotcontac"),
    html.Br(),
    html.Br(),
    html.P("Por favor selecciona una helice, en esta parte vamos a ver como cambian los contactos de una helice respecto a todas las demás."),
    html.Br(),
    dcc.Dropdown(
                id="helicec",
                options=[{'label': col, 'value': col} for col in helix_df["helice1"].unique()],
                value='TM6_PS1',style={'width':"80%",'display':'block'}),
    html.Div(id="helicecplot"),
])

## Ubicaciones y PCA

In [9]:
ubis =   html.Div([html.H4("Ubicación de regiones especificas"),
                   html.P("En esta parte vamos a comparar si hay cambios en ciertas partes de GS"),
                   html.P("Escoge que region quieres analizar"),
                   html.Br(),
                   dcc.Dropdown(
                        id="u",
                        options=[{'label': '6CT', 'value': '6ct'}],
                        value='6ct',style={'width':"45%",'display':'block'}),
                   html.Div(id="ubiplis")
                  ])

pca = html.Div([html.H4("PCA"),
                html.Br(),
                html.P("Comparemos los componentes principales de las partes membranales. Selecciona que sistemas quieres comparar"),
                dbc.Row([
                         
                dcc.Dropdown(
                    id="pcas1",
                    options=[{'label': 'Estructura Apo', 'value': 'Apo'},
                             {'label': 'Estructura Apo l166p <500', 'value': 'Mut1'},
                             {'label': 'Estructura Apo l166p >500', 'value': 'Mut2'}],
                    value='Apo',style={'width':"45%",'display':'block'}),
                         
                dcc.Dropdown(
                    id="pcas2",
                    options=[{'label': 'Estructura Apo', 'value': 'Apo'},
                             {'label': 'Estructura Apo l166p <500', 'value': 'Mut1'},
                             {'label': 'Estructura Apo l166p >500', 'value': 'Mut2'}],
                    value='Mut2',style={'width':"45%",'display':'block'})]),
                html.Div(id="plotpca")
               ])

## Cross C. y Solvente

In [10]:
ws  = pd.read_csv("assets/aposhell_wat.dat",delimiter="\s+")
ws  = ws.set_index("#Frame")

ws2  = pd.read_csv("assets/mutshell_wat.dat",delimiter="\s+")
ws2  = ws2.set_index("#Frame")

fig = go.Figure()
fig.add_traces(go.Box(x = ["Primera esfera 3.4 Å<br> Apo"]*ws.shape[0], y=ws.iloc[:,0],showlegend=False,marker_color ="forestgreen",boxmean=True))
fig.add_traces(go.Box(x = ["Primera esfera 3.4 Å<br> Mut"]*ws2.shape[0], y=ws2.iloc[:,0],showlegend=False,marker_color ="firebrick",boxmean=True))
fig.add_traces(go.Box(x = ["Segunda esfera 5.0 Å<br> Apo"]*ws.shape[0], y=ws.iloc[:,1],showlegend=False,marker_color ="forestgreen",boxmean=True))
fig.add_traces(go.Box(x = ["Segunda esfera 5.0 Å<br> Mut"]*ws2.shape[0], y=ws2.iloc[:,1],showlegend=False,marker_color ="firebrick",boxmean=True))
fig.update_layout(title="Accesibilidad al solvente dentro del sitio catalitico",title_x=0.5)
fig.update_yaxes(title="Número de aguas")  


rdf = pd.read_csv("assets/apordf_wat.dat", sep="\s+")
rdf1  = pd.read_csv("assets/mutrdf_wat.dat", sep="\s+")
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=rdf.iloc[:,0], y=rdf.iloc[:,1],
                    mode='lines',marker_color="forestgreen",
                    name='Apo'))
fig1.add_trace(go.Scatter(x=rdf1.iloc[:,0], y=rdf1.iloc[:,1],
                    mode='lines',marker_color="firebrick",
                    name='Mut'))
fig1.update_layout(title="RDF Aspárticos catilícos y Agua",title_x=0.5)
fig1.update_yaxes(title="RDF") 
fig1.update_xaxes(title="r (Å)")
aguas = html.Div([
    html.H4("Accesibilidad al solvente dentro del sitio catalitico"),
    html.Br(),
    html.P("El agua desempeña un papel esencial en el mecanismo de acción de la gamma secretasa. Cuando el residuo de ácido aspártico se encuentra desprotonado, polariza una molécula de agua, lo que da inicio a la catálisis enzimática al realizar un ataque nucleofílico sobre el carbonilo del enlace que será escindido. Este proceso forma un intermediario tetraédrico que se estabiliza cuando el oxígeno del carbonilo acepta el hidrógeno ácido proveniente del ácido aspártico catalítico, generando así un intermediario gem-diol. Posteriormente, ambos residuos aspárticos vuelven a su estado de protonación inicial, lo cual resulta en la escisión del sustrato peptídico y la liberación de productos proteolíticos."),
    html.Br(),
    html.P("La presencia de agua en este mecanismo es de gran importancia. Si el agua estuviera ausente, se espera que desempeñe un papel inactivador o inhibitorio en la actividad enzimática."),
    dcc.Graph(figure=fig,config= {'toImageButtonOptions': {'format': 'png','filename': 'custom_image','height': 500,
                                    'width': 700,'scale': 1}}),
    html.Br(),
    dcc.Graph(figure=fig1,config= {'toImageButtonOptions': {'format': 'svg','filename': 'custom_image','height': 500,
                                    'width': 700,'scale': 1}}),
])

## Asparticos y lipidos

In [11]:
asp_cat = html.Div([
    html.H4("Analisis asparticos cataliticos"),
    html.P("GS es una aspartil proteasa conformada por 4 subunidades. Las cuales son faringe anterior defectuosa (APH1), presenilina (PS), potenciador de presenilina 2 (PEN2) y nicastrina (NCST), a su vez nicastrina se puede subdividir en lóbulo mayor y lóbulo menor"),
    html.P("PSEN es la subunidad catalítica de GS, se compone de 9 hélices transmembranales (TM). Los aspárticos catalíticos se encuentran en las hélices TM6 y TM7. Debido a una endoproteólisis necesaria para la maduración del complejo, PSEN está divido en sus dominios N y C terminales, los cuales se componen de los segmentos TM1-TM6 y TM7-TM9 respectivamente."),
    html.P("Al igual que otras aspartil proteasas, GS hidroliza a los sustratos mediante un mecanismo ácido-base que involucra los dos residuos de aspárticos y una molécula de agua. De manera general lo que ocurre es un ataque nucleofílico favorecido por la molécula de agua que polariza a un carboxilato. Se ha determinado mediante métodos de dinámica molecular que el aspártico catalítico 385 es aquel que se protona de manera preferente a pH fisiológico y en presencia del sustrato"),
    dbc.Row([
        html.Span("Grafiquemos la distancia de los asparticos cataliticos como :  ",style={'display':'block'}),
        dcc.Dropdown(
                id="dtplot",
                options=[{'label': 'Histograma', 'value': 'hist'},
                         {'label': 'Evolución temporal', 'value': 'tiempo'},
                         {'label': 'Histograma 2D', 'value': 'hist2D'}],
                value='hist',style={'width':"70%",'display':'block'})]),
        
    html.Div(id="dplot")
])

lipyphilic = html.Div([
    html.H4("Analisis de integridad de la membrana"),
    html.H1("Ingresa informacion de Parametro de orden y Perfil de densidad"),
    dbc.Row([
        html.Span("Selecciona el analisis de membrana que te gustaria ver :  ",style={'display':'block'}),
        dcc.Dropdown(
                id="lipy",
                options=[{'label': 'Parametro de orden', 'value': 'order'},
                         {'label': 'Perfil de densidad', 'value': 'mass'}],
                value='order',style={'width':"70%",'display':'block'})]),
        
    html.Div(id="liplot")
])

# Inicio de la app

In [12]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.SIMPLEX],
               meta_tags=[{'name': 'viewport',
                            'content': 'width=device-width, initial-scale=1.0'}])   
app.layout = html.Div([
    dcc.Location(id="url"),
    sidebar,
    content,
    dcc.Store(id='side_click'),
    dcc.Store(id="num-format"),
    dcc.Store(id="img-format"),
])

In [13]:
@app.callback(
    [
        Output("sidebar", "style"),
        Output("page-content", "style"),
        Output("side_click", "data"),
    ],

    [Input("btn_sidebar", "n_clicks")],
    [
        State("side_click", "data"),
    ]
)
def toggle_sidebar(n, nclick):
    if n:
        if nclick == "SHOW":
            sidebar_style = SIDEBAR_HIDEN
            content_style = CONTENT_STYLE1
            cur_nclick = "HIDDEN"
        else:
            sidebar_style = SIDEBAR_STYLE
            content_style = CONTENT_STYLE
            cur_nclick = "SHOW"
    else:
        sidebar_style = SIDEBAR_STYLE
        content_style = CONTENT_STYLE
        cur_nclick = 'SHOW'

    return sidebar_style, content_style, cur_nclick

# Funciones
##  Barside

In [14]:
#  Aca definimos las funciones o callback con el orden:
# * URL sidebar
# * Store img format
# * Volcano MA plot y el histograma
@app.callback(
    Output("page-content", "children"),
    [Input("url", "pathname")]
)


def render_page_content(pathname):
    if pathname == "/":
        return [encabezado,html.Hr(),Intro]
    elif pathname == "/rmsd":
        return [encabezado,p_container]
    elif pathname == "/rmsf":
        return [encabezado, html.Hr(), rmsf]
    elif pathname == "/sec-estruc":
        return [encabezado, html.Hr(), ss]
    elif pathname == "/helanal":
        return [encabezado, html.Hr(), tilt]
    elif pathname == "/contact":
        return [encabezado, html.Hr(), contete]
    elif pathname == "/ubicacion":
        return [encabezado, html.Hr(), ubis]
    elif pathname == "/pca":
        return [encabezado, html.Hr(), pca]
    elif pathname == "/h2o":
        return [encabezado, html.Hr(), aguas]
    elif pathname == "/asp-cat":
        return [encabezado, html.Hr(), asp_cat]
    elif pathname == "/lipyphilic":
        return [encabezado, html.Hr(), lipyphilic]
    else:
        return html.Div([
            html.H1("404: Not found", className="bg-danger text-white"),
            html.H1(f"The pathname {pathname} was not recognised...") ])
@app.callback(
    Output(component_id="img-format", component_property='data'),
    Input(component_id="format-dw-img", component_property="value"),
)
def download_img_format(f):
    svg = {'toImageButtonOptions': {'format': 'svg','filename': 'custom_image','height': 500,
                                    'width': 700,'scale': 1},
          'displaylogo': False}
    png = {'toImageButtonOptions': {'format': 'png', 'filename': 'custom_image','height': 500,
                                   'width': 700,'scale': 1 },
          'displaylogo': False}

    jpeg = {'toImageButtonOptions': {'format': 'jpeg', 'filename': 'custom_image','height': 500,
                                    'width': 700,'scale': 1},
           'displaylogo': False}

    webp = {'toImageButtonOptions': {'format': 'webp','filename': 'custom_image','height': 500,
                                    'width': 700,'scale': 1},
           'displaylogo': False}
    if  f=="svg":
        return svg
    elif f=="png":
        return png
    elif f=="jpeg":
        return jpeg
    elif f=="webp":
        return webp

## RMSD

In [15]:
#********************************R M S D   P O R  E S T R U C T U R A*****************************************************   
@app.callback(
    Output("plots-rmsd", "children"),
    Input(component_id='tipormsd', component_property='value'),         #Estructura deseada
    Input(component_id='estruc', component_property='value'),         #Estructura deseada
    Input(component_id='sub1', component_property='value'),         #Estructura deseada
    Input(component_id="img-format", component_property='data'),
)

def update_rmsd(tipo,est,sub1,f): 
    if tipo == "sis":
        if est=="nat":
            time = nat_rmsd[0][:,0]/100 
            n = go.Figure()
            n.add_traces(go.Scatter(x=time,y=nat_rmsd[0][:,1],marker_color="indianred"))
            n.update_layout(title="RMSD NCT",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=30))
            n.update_xaxes(title="Tiempo (ns)")
            n.update_yaxes(title="RMSD (Å)")

            s = go.Figure()
            s.add_traces(go.Scatter(x=time,y=nat_rmsd[1][:,1],marker_color="goldenrod"))
            s.update_layout(title="RMSD PS1",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=30))
            s.update_xaxes(title="Tiempo (ns)")
            s.update_yaxes(title="RMSD (Å)")

            a = go.Figure()
            a.add_traces(go.Scatter(x=time,y=nat_rmsd[2][:,1],marker_color="steelblue"))
            a.update_layout(title="RMSD APH1 ",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=30))
            a.update_xaxes(title="Tiempo (ns)")
            a.update_yaxes(title="RMSD (Å)")

            p = go.Figure()
            p.add_traces(go.Scatter(x=time,y=nat_rmsd[3][:,1],marker_color="olivedrab"))
            p.update_layout(title="RMSD PEN2 ",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=30))
            p.update_xaxes(title="Tiempo (ns)")
            p.update_yaxes(title="RMSD (Å)")
            fig = go.Figure()
            fig.add_traces(go.Scatter(x=time,y=nat_rmsd[4][:,1],marker_color="black"))
            fig.update_layout(title="RMSD γ-secretasa Nativa",title_x=0.5,yaxis_range=[0.7,5],font=dict(family="Courier New, monospace",size=30))
            fig.update_xaxes(title="Tiempo (ns)")
            fig.update_yaxes(title="RMSD (Å)")

            plot= html.Div([
                dbc.Row([
                    dbc.Col([
                        dcc.Graph(figure=n,config=f),
                        dcc.Graph(figure=s,config=f)],width={"size":6,"offset":0}),

                dbc.Col([
                    dcc.Graph(figure=a,config=f),
                    dcc.Graph(figure=p,config=f)],width={"size":6,"offset":0}),

                dbc.Col([
                    dcc.Graph(figure=fig,config=f)]),
                ])])
        
        if est=="mut":
            time = mut_rmsd[0][:,0]/100
            n = go.Figure()
            n.add_traces(go.Scatter(x=time,y=mut_rmsd[0][:,1],marker_color="indianred"),)
            n.update_layout(title="RMSD NCT",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=30))
            n.update_xaxes(title="Tiempo (ns)")
            n.update_yaxes(title="RMSD (Å)")

            s = go.Figure()
            s.add_traces(go.Scatter(x=time,y=mut_rmsd[1][:,1],marker_color="goldenrod"))
            s.update_layout(title="RMSD PS1",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=30))
            s.update_xaxes(title="Tiempo (ns)")
            s.update_yaxes(title="RMSD (Å)")

            a = go.Figure()
            a.add_traces(go.Scatter(x=time,y=mut_rmsd[2][:,1],marker_color="steelblue"))
            a.update_layout(title="RMSD APH1 ",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=30))
            a.update_xaxes(title="Tiempo (ns)")
            a.update_yaxes(title="RMSD (Å)")

            p = go.Figure()
            p.add_traces(go.Scatter(x=time,y=mut_rmsd[3][:,1],marker_color="olivedrab"))
            p.update_layout(title="RMSD PEN2 ",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=30))
            p.update_xaxes(title="Tiempo (ns)")
            p.update_yaxes(title="RMSD (Å)")

            fig = go.Figure()
            fig.add_traces(go.Scatter(x=time,y=mut_rmsd[4][:,1],marker_color="black"))
            fig.update_layout(title="RMSD γ-secretasa Nativa-L166P",title_x=0.5,yaxis_range=[0.7,5],font=dict(family="Courier New, monospace",size=30))
            fig.update_xaxes(title="Tiempo (ns)")
            fig.update_yaxes(title="RMSD (Å)")

            plot= html.Div([
                dbc.Row([
                    dbc.Col([
                        dcc.Graph(figure=n,config=f),
                        dcc.Graph(figure=s,config=f)],width={"size":6,"offset":0}),

                dbc.Col([
                    dcc.Graph(figure=a,config=f),
                    dcc.Graph(figure=p,config=f)],width={"size":6,"offset":0}),

                dbc.Col([
                    dcc.Graph(figure=fig,config=f)]),
                ])])

###########################################################################################################################################################
    elif tipo =="comp":  
        window=50
        if sub1=="ncst":
            n = go.Figure()
            n.add_traces(go.Scatter(x=nat_rmsd[0][:,0]/100 ,
                                    y=nat_rmsd[0][:,1],marker_color="indianred"))
            n.update_layout(title="RMSD NCT Estructura Nativa",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=30))
            n.update_xaxes(title="Tiempo (ns)")
            n.update_yaxes(title="RMSD (Å)")

            m = go.Figure()
            m.add_traces(go.Scatter(x=mut_rmsd[0][:,0]/100 ,
                                    y=mut_rmsd[0][:,1],marker_color="indianred"))
            m.update_layout(title="RMSD NCT Estructura Nativa-L166P",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=30))
            m.update_xaxes(title="Tiempo (ns)")
            m.update_yaxes(title="RMSD (Å)")
            
            s = go.Figure()
            s.add_traces(go.Scatter(x=mut_ncst[0][:,0]/100,y=np.convolve(mut_ncst[0][:,1], np.ones(window)/window, mode='same'),marker_color="blue",name="Primer frame"))
            s.add_traces(go.Scatter(x=mut_ncst[1][:,0]/100,y=np.convolve(mut_ncst[1][:,1], np.ones(window)/window, mode='same'),marker_color="goldenrod",name="+Rep Mut"))
            s.add_traces(go.Scatter(x=mut_ncst[2][:,0]/100,y=np.convolve(mut_ncst[2][:,1], np.ones(window)/window, mode='same'),marker_color="red",name="+Rep Apo"))
            s.update_layout(title="RMSD NCT",title_x=0.5,font=dict(family="Courier New, monospace",size=30))
            s.update_xaxes(title="Tiempo (ns)")
            s.update_yaxes(title="RMSD (Å)")
            plot= html.Div([
                dbc.Row([
                    dbc.Col([
                        dcc.Graph(figure=n,config=f)],width={"size":6,"offset":0}),

                dbc.Col([
                    dcc.Graph(figure=m,config=f)],width={"size":6,"offset":0}),
                ]),
                dcc.Graph(figure=s,config=f)])
            
        if sub1=="psn1":
            n = go.Figure()
            n.add_traces(go.Scatter(x=nat_rmsd[1][:,0]/100 ,
                                    y=nat_rmsd[1][:,1],marker_color="goldenrod"))
            n.update_layout(title="RMSD PS1 Estructura Nativa",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=30))
            n.update_xaxes(title="Tiempo (ns)")
            n.update_yaxes(title="RMSD (Å)")
            
            m = go.Figure()
            m.add_traces(go.Scatter(x=mut_rmsd[1][:,0]/100 ,
                                    y=mut_rmsd[1][:,1],marker_color="goldenrod"))
            m.update_layout(title="RMSD PS1 Estructura Nativa-L166P",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=30))
            m.update_xaxes(title="Tiempo (ns)")
            m.update_yaxes(title="RMSD (Å)")
            s = go.Figure()
            s.add_traces(go.Scatter(x=mut_psen[0][:,0]/100,y=np.convolve(mut_psen[0][:,1], np.ones(window)/window, mode='same'),marker_color="blue",name="Primer frame"))
            s.add_traces(go.Scatter(x=mut_psen[1][:,0]/100,y=np.convolve(mut_psen[1][:,1], np.ones(window)/window, mode='same'),marker_color="goldenrod",name="+Rep Mut"))
            s.add_traces(go.Scatter(x=mut_psen[2][:,0]/100,y=np.convolve(mut_psen[2][:,1], np.ones(window)/window, mode='same'),marker_color="red",name="+Rep Apo"))
            s.update_layout(title="RMSD PS1",title_x=0.5,font=dict(family="Courier New, monospace",size=30))
            s.update_xaxes(title="Tiempo (ns)")
            s.update_yaxes(title="RMSD (Å)")
            plot= html.Div([
                dbc.Row([
                    dbc.Col([
                        dcc.Graph(figure=n,config=f)],width={"size":6,"offset":0}),

                dbc.Col([
                    dcc.Graph(figure=m,config=f)],width={"size":6,"offset":0}),
                ]),dcc.Graph(figure=s,config=f)])
            
        if sub1=="aph1":
            n = go.Figure()
            n.add_traces(go.Scatter(x=nat_rmsd[2][:,0]/100 ,
                                    y=nat_rmsd[2][:,1],marker_color="steelblue"))
            n.update_layout(title="RMSD APH1 Estructura Nativa",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=30))
            n.update_xaxes(title="Tiempo (ns)")
            n.update_yaxes(title="RMSD (Å)")
            
            m = go.Figure()
            m.add_traces(go.Scatter(x=mut_rmsd[2][:,0]/100 ,
                                    y=mut_rmsd[2][:,1],marker_color="steelblue"))
            m.update_layout(title="RMSD APH1 Estructura Nativa-L166P",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=30))
            m.update_xaxes(title="Tiempo (ns)")
            m.update_yaxes(title="RMSD (Å)")
            s = go.Figure()
            s.add_traces(go.Scatter(x=mut_aph1[0][:,0]/100,y=np.convolve(mut_aph1[0][:,1], np.ones(window)/window, mode='same'),marker_color="blue",name="Primer frame"))
            s.add_traces(go.Scatter(x=mut_aph1[1][:,0]/100,y=np.convolve(mut_aph1[1][:,1], np.ones(window)/window, mode='same'),marker_color="goldenrod",name="+Rep Mut"))
            s.add_traces(go.Scatter(x=mut_aph1[2][:,0]/100,y=np.convolve(mut_aph1[2][:,1], np.ones(window)/window, mode='same'),marker_color="red",name="+Rep Apo"))
            s.update_layout(title="RMSD APH1",title_x=0.5,font=dict(family="Courier New, monospace",size=30))
            s.update_xaxes(title="Tiempo (ns)")
            s.update_yaxes(title="RMSD (Å)")
            plot= html.Div([
                dbc.Row([
                    dbc.Col([
                        dcc.Graph(figure=n,config=f)],width={"size":6,"offset":0}),

                dbc.Col([
                    dcc.Graph(figure=m,config=f)],width={"size":6,"offset":0}),
                ]),dcc.Graph(figure=s,config=f)])
        if sub1=="pen2":
            n = go.Figure()
            n.add_traces(go.Scatter(x=nat_rmsd[3][:,0]/100 ,
                                    y=nat_rmsd[3][:,1],marker_color="olivedrab"))
            n.update_layout(title="RMSD PEN2 Estructura Nativa",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=30))
            n.update_xaxes(title="Tiempo (ns)")
            n.update_yaxes(title="RMSD (Å)")
            
            m = go.Figure()
            m.add_traces(go.Scatter(x=mut_rmsd[3][:,0]/100 ,
                                    y=mut_rmsd[3][:,1],marker_color="olivedrab"))
            m.update_layout(title="RMSD PEN2 Estructura Nativa-L166P",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=30))
            m.update_xaxes(title="Tiempo (ns)")
            m.update_yaxes(title="RMSD (Å)")
            s = go.Figure()
            s.add_traces(go.Scatter(x=mutpen2[0][:,0]/100,y=np.convolve(mutpen2[0][:,1], np.ones(window)/window, mode='same'),marker_color="blue",name="Primer frame"))
            s.add_traces(go.Scatter(x=mutpen2[1][:,0]/100,y=np.convolve(mutpen2[1][:,1], np.ones(window)/window, mode='same'),marker_color="goldenrod",name="+Rep Mut"))
            s.add_traces(go.Scatter(x=mutpen2[2][:,0]/100,y=np.convolve(mutpen2[2][:,1], np.ones(window)/window, mode='same'),marker_color="red",name="+Rep Apo"))
            s.update_layout(title="RMSD PEN2",title_x=0.5,font=dict(family="Courier New, monospace",size=30))
            s.update_xaxes(title="Tiempo (ns)")
            s.update_yaxes(title="RMSD (Å)")
            plot= html.Div([
                dbc.Row([
                    dbc.Col([
                        dcc.Graph(figure=n,config=f)],width={"size":6,"offset":0}),
                    dbc.Col([dcc.Graph(figure=m,config=f)],width={"size":6,"offset":0}),
                ]),dcc.Graph(figure=s,config=f)])
            
        if sub1=="all":
            n = go.Figure()
            n.add_traces(go.Scatter(x=nat_rmsd[4][:,0]/100 ,
                                    y=nat_rmsd[4][:,1],marker_color="black"))
            n.update_layout(title="RMSD Estructura Nativa",title_x=0.5,yaxis_range=[0.7,5],font=dict(family="Courier New, monospace",size=15))
            n.update_xaxes(title="Tiempo (ns)")
            n.update_yaxes(title="RMSD (Å)")
            
            m = go.Figure()
            m.add_traces(go.Scatter(x=mut_rmsd[4][:,0]/100 ,
                                    y=mut_rmsd[4][:,1],marker_color="black"))
            m.update_layout(title="RMSD Estructura Nativa-L166P",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=15))
            m.update_xaxes(title="Tiempo (ns)")
            m.update_yaxes(title="RMSD (Å)")
            s = go.Figure()
            s.add_traces(go.Scatter(x=mut1_gamma[0][:,0]/100,y=np.convolve(mut1_gamma[0][:,1], np.ones(window)/window, mode='same'),marker_color="blue",name="Primer frame"))
            s.add_traces(go.Scatter(x=mut1_gamma[1][:,0]/100,y=np.convolve(mut1_gamma[1][:,1], np.ones(window)/window, mode='same'),marker_color="goldenrod",name="+Rep Mut"))
            s.add_traces(go.Scatter(x=mut1_gamma[2][:,0]/100,y=np.convolve(mut1_gamma[2][:,1], np.ones(window)/window, mode='same'),marker_color="red",name="+Rep Apo"))
            s.update_layout(title="RMSD Complejo Enzimatico",title_x=0.5,font=dict(family="Courier New, monospace",size=30))
            s.update_xaxes(title="Tiempo (ns)")
            s.update_yaxes(title="RMSD (Å)")
            plot= html.Div([
                dbc.Row([
                    dbc.Col([
                        dcc.Graph(figure=n,config=f)],width={"size":6,"offset":0}),

                dbc.Col([dcc.Graph(figure=m,config=f)],width={"size":6,"offset":0}),
                ]),dcc.Graph(figure=s,config=f)])
    return (plot)

## RMSF

In [16]:
#********************************R M S F   P O R  S U B U NI D A D *****************************************************   
@app.callback(
    Output("plots-rmsf", "children"),
    Input(component_id="img-format", component_property='data'),
    Input(component_id="format-dw-pot", component_property="value"),
)
def update_rmsf(f,ff):
    if  ff=="amber":
        aa = df[df["Sistema"]=="Estructura Nativa"]["Subunidad"].values+": "+df[df["Sistema"]=="Estructura Nativa"]["Aminoacido"].values+"-"+df[df["Sistema"]=="Estructura Nativa"]["Numeracion"].values.astype("str")
        aa1 = df[df["Sistema"]=="Complejo Enzima-Sustrato"]["Subunidad"].values+": "+df[df["Sistema"]=="Complejo Enzima-Sustrato"]["Aminoacido"].values+"-"+df[df["Sistema"]=="Complejo Enzima-Sustrato"]["Numeracion"].values.astype("str")
    elif ff=="ab":
        aa = df[df["Sistema"]=="Estructura Nativa"]["Subunidad"].values+": "+df[df["Sistema"]=="Estructura Nativa"]["Aminoacido"].values+"-"+df[df["Sistema"]=="Estructura Nativa"]["Corregida"].values.astype("str")
        aa1 = df[df["Sistema"]=="Complejo Enzima-Sustrato"]["Subunidad"].values+": "+df[df["Sistema"]=="Complejo Enzima-Sustrato"]["Aminoacido"].values+"-"+df[df["Sistema"]=="Complejo Enzima-Sustrato"]["Corregida"].values.astype("str")
    elif ff=="c83":
        aa = df[df["Sistema"]=="Estructura Nativa"]["Subunidad"].values+": "+df[df["Sistema"]=="Estructura Nativa"]["Aminoacido"].values+"-"+df[df["Sistema"]=="Estructura Nativa"]["Corregida2"].values.astype("str")
        aa1 = df[df["Sistema"]=="Complejo Enzima-Sustrato"]["Subunidad"].values+": "+df[df["Sistema"]=="Complejo Enzima-Sustrato"]["Aminoacido"].values+"-"+df[df["Sistema"]=="Complejo Enzima-Sustrato"]["Corregida2"].values.astype("str")
    n = go.Figure()
    n.add_traces(go.Scatter(x=aa, y=nat_rmsf[:,1],marker_color="black"))
    n.update_layout(title="RMSF Estructura Apo",title_x=0.5,yaxis_range=[-0.5,9.5],font=dict(family="Courier New, monospace",size=15))
    n.add_vrect(x0=aa1[0], x1=aa[666], fillcolor="indianred", opacity=0.25, line_width=0)
    n.add_vrect(x0=aa1[666], x1=aa[977], fillcolor="gold", opacity=0.25, line_width=0)
    n.add_vrect(x0=aa1[977], x1=aa[1221], fillcolor="steelblue", opacity=0.25, line_width=0)
    n.add_vrect(x0=aa1[1221], x1=aa[-1], fillcolor="olivedrab", opacity=0.25, line_width=0)
    n.update_xaxes(title="Número de residuo")
    n.update_yaxes(title="RMSF (Å)")
    ##############################################################################################
    s = go.Figure()
    s.add_traces(go.Scatter(x=aa, y=mut_rmsf[:,1],marker_color="black"))
    s.update_layout(title="RMSF Estructura Apo L166P ",title_x=0.5,yaxis_range=[-0.5,9.5],font=dict(family="Courier New, monospace",size=15))
    s.add_vrect(x0=aa[0], x1=aa[666], fillcolor="indianred", opacity=0.25, line_width=0)
    s.add_vrect(x0=aa[666], x1=aa[977], fillcolor="gold", opacity=0.25, line_width=0)
    s.add_vrect(x0=aa[977], x1=aa[1221], fillcolor="steelblue", opacity=0.25, line_width=0)
    s.add_vrect(x0=aa[1221], x1=aa[-1], fillcolor="olivedrab", opacity=0.25, line_width=0)
    s.update_xaxes(title="Número de residuo")
    s.update_yaxes(title="RMSF (Å)")
    
    fig = go.Figure()
    fig.update_layout(title="RMSF Sistemas de GS",title_x=0.5,yaxis_range=[-0.5,9.5],font=dict(family="Courier New, monospace",size=15))
    fig.add_traces(go.Scatter(x=aa, y=nat_rmsf[:,1],marker_color="forestgreen",name="Estructura Nativa"))
    fig.add_traces(go.Scatter(x=aa, y=mut_rmsf[:,1],marker_color="firebrick",name="Estructura Mutada"))
    fig.add_traces(go.Scatter(x=aa,y=mut_rmsf2["AtomicFlx"], marker_color="blue",name="Replica Mutada"))
    fig.add_vrect(x0=aa[0], x1=aa[666], fillcolor="indianred", opacity=0.25, line_width=0)
    fig.add_vrect(x0=aa[666], x1=aa[977], fillcolor="gold", opacity=0.25, line_width=0)
    fig.add_vrect(x0=aa[977], x1=aa[1221], fillcolor="steelblue", opacity=0.25, line_width=0)
    fig.add_vrect(x0=aa[1221], x1=aa[1321], fillcolor="olivedrab", opacity=0.25, line_width=0)
    fig.update_xaxes(title="Número de residuo")
    fig.update_yaxes(title="RMSF (Å)")
    
    plot= html.Div([
    dbc.Row([
        dbc.Col([
            dcc.Graph(figure=n,config=f)],width={"size":6,"offset":0}),

    dbc.Col([
        dcc.Graph(figure=s,config=f)],width={"size":6,"offset":0}),
    html.Br(),
    dbc.Row(dcc.Graph(figure=fig,config=f))
    ])])
    return (plot)

## Estructura secundaria

In [17]:
#******************************** E S T R U C T U R A   S E C U N D A R I A *****************************************************   
@app.callback(
    Output("ss-div", "children"),
    Input(component_id='ss-sub', component_property='value'),         #Estructura deseada
    Input(component_id="img-format", component_property='data'),
    Input(component_id="format-dw-pot", component_property="value"),
)
def ssk(kind,f,ff):
    if  ff=="amber":
        aa = df[df["Sistema"]=="Estructura Nativa"]["Subunidad"].values+": "+df[df["Sistema"]=="Estructura Nativa"]["Aminoacido"].values+"-"+df[df["Sistema"]=="Estructura Nativa"]["Numeracion"].values.astype("str")
        aa1 = df[df["Sistema"]=="Complejo Enzima-Sustrato"]["Subunidad"].values+": "+df[df["Sistema"]=="Complejo Enzima-Sustrato"]["Aminoacido"].values+"-"+df[df["Sistema"]=="Complejo Enzima-Sustrato"]["Numeracion"].values.astype("str")
    elif ff=="ab":
        aa = df[df["Sistema"]=="Estructura Nativa"]["Subunidad"].values+": "+df[df["Sistema"]=="Estructura Nativa"]["Aminoacido"].values+"-"+df[df["Sistema"]=="Estructura Nativa"]["Corregida"].values.astype("str")
        aa1 = df[df["Sistema"]=="Complejo Enzima-Sustrato"]["Subunidad"].values+": "+df[df["Sistema"]=="Complejo Enzima-Sustrato"]["Aminoacido"].values+"-"+df[df["Sistema"]=="Complejo Enzima-Sustrato"]["Corregida"].values.astype("str")
    elif ff=="c83":
        aa = df[df["Sistema"]=="Estructura Nativa"]["Subunidad"].values+": "+df[df["Sistema"]=="Estructura Nativa"]["Aminoacido"].values+"-"+df[df["Sistema"]=="Estructura Nativa"]["Corregida2"].values.astype("str")
        aa1 = df[df["Sistema"]=="Complejo Enzima-Sustrato"]["Subunidad"].values+": "+df[df["Sistema"]=="Complejo Enzima-Sustrato"]["Aminoacido"].values+"-"+df[df["Sistema"]=="Complejo Enzima-Sustrato"]["Corregida2"].values.astype("str")
        
    if kind=="nat":
        ss_ss = pd.read_csv('assets/ssecondary_nat.dat.sum',sep="\s+")
        imagen = "Estructura secundaria_nat.svg"
        title="Estructura Secundaria Estructura Nativa"
        amino =aa
    elif kind=="mut":
        ss_ss = pd.read_csv('assets/ssecondary_mut.dat.sum',sep="\s+")
        imagen = "Estructura secundaria_mut.svg"
        title="Estructura Secundaria Estructura Nativa L166P"
        amino =aa
        
    #Este chunk de codigo es para dejar solo la estructura mas representativa en las laminas beta
    ss_ss["Fracc" ]= ss_ss["Para"].values+ss_ss["Anti"].values+ss_ss["3-10"].values+ss_ss["Alpha"].values+ss_ss["Pi"].values+ss_ss["Turn"].values+ss_ss["Bend"].values
    mn = ss_ss[ss_ss["Fracc"]>1.5].iloc[:,1:3].values.argmin(axis=1) +1
    ind = ss_ss[ss_ss["Fracc"]>1.5].index
    for i in range(len(mn)):
        ss_ss.iloc[ind[i],mn[i]] = 0
    ss_ss["Fracc" ]= ss_ss["Para"].values+ss_ss["Anti"].values+ss_ss["3-10"].values+ss_ss["Alpha"].values+ss_ss["Pi"].values+ss_ss["Turn"].values+ss_ss["Bend"].values
    #aca acaba el chunk
    
    colores=["steelblue","midnightblue","maroon","firebrick","darkgoldenrod","palegoldenrod", "beige"]
    tipos =['Para',"Anti","3-10","Alpha","Pi","Turn", "Bend"]
    fig = go.Figure()
    for i in range(len(tipos)):
        fig.add_traces(go.Bar(x=amino, y=ss_ss[tipos[i]],name=tipos[i],
                             marker_color=colores[i]))
    fig.update_layout(title_x=0.5,title_text=title,barmode="stack",width=2000,height=400,)
    fig.update_xaxes(title="Residuos")
    fig.update_yaxes(title="Probabilidad")
    
    plot=html.Div([dbc.Row(dcc.Graph(figure=fig,config=f)),
                   html.Br(),html.Br(),
                   dbc.Col(html.Img(src="assets/"+imagen,style={ 'width':'100%','height':'100%'},className="d-flex align-self-center"))
                  ])
    return plot

## HELANAL

In [18]:
#******************************** H E L A N A L  **********************************************************   
@app.callback(
    Output("pcaplot", "children"),
    Input(component_id='helanalanal', component_property='value'),         #Estructura deseada
    Input(component_id="img-format", component_property='data'),
)
def dist_cat(tipo,f):  
    if tipo=="tilt": 
        plot= html.Div([
        html.Img(src="assets/Tilt_dots.png",style={'height':'190%', 'width':'190%'},className="d-flex align-self-center"),
        html.Br(),
        html.Img(src="assets/Tilt.svg",style={'height':'90%', 'width':'90%'},className="d-flex align-self-center"),
        ])
    elif tipo =="2he":
        plot= html.Div([
        html.Img(src="assets/helices_angulo.png",style={'height':'90%', 'width':'90%'},className="d-flex align-self-center"),
        html.Br(),])
    elif tipo=="kinky":
            plot= html.Div([
        html.Img(src="assets/kinks.png",style={'height':'90%', 'width':'90%'},className="d-flex align-self-center"),
        html.Br()])
    return plot
        

## Contactos membranales

In [19]:
#******************************** C O N T A C T O S  **********************************************************   
@app.callback(
    Output("plotcontac", "children"),
    Input(component_id='contact1', component_property='value'),         #Estructura deseada
    Input(component_id='contact2', component_property='value'),         #Estructura deseada
    Input(component_id="img-format", component_property='data'),
)
def contacmat(pcas1,pcas2,f): 
    if pcas1 =="Apo":
        mat1 = apo_cont
    elif pcas1 =="Mut":
        mat1 = mut_cont
    if pcas2 =="Apo":
        mat2 = apo_cont
    elif pcas2 =="Mut":
        mat2 = mut_cont

    resta = mat1-mat2
    resta2 = mat2-mat1
    resta = np.where(resta <= 0, 0, resta)
    resta2 = np.where(resta2 <= 0, 0, resta2)
    
    fig = make_subplots(rows=1,cols=2,subplot_titles=["Contactos Sistema "+pcas1,"Contactos Sistema "+pcas2 ])
    fig.add_trace(go.Heatmap(
                       z=mat1,
                       x=aa[635:-1],
                       y=aa[635:-1],colorscale="Aggrnyl"),row=1,col=1)

    fig.add_trace(go.Heatmap(
                       z=mat2,
                       x=aa[635:-1],
                       y=aa[635:-1],colorscale="Aggrnyl"),row=1,col=2)

    fig.update_layout(height=1200, width=2000, title_text="Contactos membranales",title_x=0.5)
    fig.update_xaxes(tickangle=-90)
    
    fig2 = make_subplots(rows=1,cols=2,subplot_titles=["Si en "+pcas1+" no en "+pcas2, "Si en "+pcas2+" no en "+pcas1])
    fig2.add_trace(go.Heatmap(
                       z=np.triu(resta),
                       x=aa[635:-1],
                       y=aa[635:-1],colorscale="Aggrnyl"),row=1,col=1)

    fig2.add_trace(go.Heatmap(
                       z=np.triu(resta2),
                       x=aa[635:-1],
                       y=aa[635:-1],colorscale="Aggrnyl"),row=1,col=2)

    fig2.update_layout(height=1200, width=2000, title_text="Contactos membranales",title_x=0.5)
    fig2.update_xaxes(tickangle=-90)
        
    plotconta = html.Div([
        dbc.Row(dcc.Graph(figure=fig,config=f)),
        html.Br(),
        html.Br(),
        dbc.Row(dcc.Graph(figure=fig2,config=f))])
    return plotconta


#******************************** C O N T A C T O S  **********************************************************   
@app.callback(
    Output("helicecplot", "children"),
    Input(component_id='helicec', component_property='value'),
    Input(component_id="img-format", component_property='data'),
)
def helix_conana(helicec,f): 
    fig3 = px.bar(helix_df[helix_df["helice1"]==helicec], x='Contactos', y='Fracción',color="Sistema", barmode="group",
              color_discrete_sequence=['forestgreen','firebrick'],title="Contactos con "+helicec)
    
    plothelicon =html.Div([
        dbc.Row(dcc.Graph(figure=fig3,config=f))])
    return plothelicon

## Ubicacion de regiones 

In [20]:
#******************************** U B I C A C I O N   **********************************************************   
@app.callback(
    Output("ubiplis", "children"),
    Input(component_id='u', component_property='value'),
    Input(component_id="img-format", component_property='data'),
)
def ubiporfis(u,f): 
    if u=="6ct":
        fig = go.Figure()
        fig.add_traces(go.Violin(y=asp1.iloc[:,1],name="Asp Apo",meanline_visible=True,line_color="forestgreen"))
        fig.add_traces(go.Violin(y=asp.iloc[:,1],name="Asp Mut",meanline_visible=True,line_color="firebrick"))

        fig.add_traces(go.Violin(y=p311.iloc[:,1],name="P31 lower Apo",meanline_visible=True,line_color="forestgreen"))
        fig.add_traces(go.Violin(y=p31.iloc[:,1],name="P31 lower Mut",meanline_visible=True,line_color="firebrick"))

        fig.add_traces(go.Violin(y=popc1.iloc[:,1],name="POPC Apo",meanline_visible=True,line_color="forestgreen"))
        fig.add_traces(go.Violin(y=popc.iloc[:,1],name="POPC Mut",meanline_visible=True,line_color="firebrick"))

        fig.add_traces(go.Violin(y=tm21.iloc[:,1],name="Loop TM2-TM3 Apo",meanline_visible=True,line_color="forestgreen"))
        fig.add_traces(go.Violin(y=tm2.iloc[:,1],name="Loop TM2-TM3 Mut",meanline_visible=True,line_color="firebrick"))

        fig.update_xaxes(title="Punto de referencia",tickangle=-90)
        fig.update_yaxes(title="Distancia (Å)")
        fig.update_layout(title="Distancia de centros de masa <br> Helice 6A  - ", title_x=0.5)
        fig2 = go.Figure()
        fig2.add_traces(go.Violin(y=difz31,name="P31 lower Apo",meanline_visible=True,line_color="forestgreen"))
        fig2.add_traces(go.Violin(y=difz311,name="P31 lower Mut",meanline_visible=True,line_color="firebrick"))

        fig2.add_traces(go.Violin(y=difzpopc,name="POPC Apo",meanline_visible=True,line_color="forestgreen"))
        fig2.add_traces(go.Violin(y=difzpopc1,name="POPC Mut",meanline_visible=True,line_color="firebrick"))


        fig2.update_layout(title="Distancia de centros de masa <br> Helice 6A  - ",title_x=0.5)
        fig2.update_yaxes(title="Distancia (Å)")
    plotubiubi =html.Div([
        dbc.Row(dcc.Graph(figure=fig,config=f)),
         dbc.Row(dcc.Graph(figure=fig2,config=f)),
    ])
    return plotubiubi

## PCA

In [21]:
#******************************** P C A  **********************************************************   
@app.callback(
    Output("plotpca", "children"),
    Input(component_id='pcas1', component_property='value'),         #Estructura deseada
    Input(component_id='pcas2', component_property='value'),         #Estructura deseada
)
def dist_cat(pcas1,pcas2): 
    plotpca = html.Div([
        html.B("γ-Secretasa "+pcas1),
        dbc.Row([
            dbc.Col([
                html.B("PC1"),
                html.Img(src="assets/pca1"+pcas1+".gif",style={'height':'130%', 'width':'95%'},className="d-flex align-self-center")]),
            dbc.Col([
                html.B("PC2"),
                html.Img(src="assets/pca2"+pcas1+".gif",style={'height':'130%', 'width':'95%'},className="d-flex align-self-center")]),
            dbc.Col([
                html.B("PC3"),
                html.Img(src="assets/pca3"+pcas1+".gif",style={'height':'130%', 'width':'95%'},className="d-flex align-self-center")]), 
            html.Br(),
            html.Br(),
            html.Br(),
            html.Br(),
            html.Br(),
            html.Br(),
            html.Br(),
            html.Br(),
            html.Br(),
            html.Br(),
            html.Br(),
            html.Br(),
            html.B("γ-Secretasa "+pcas2),
        dbc.Row([
            dbc.Col([
                html.B("PC1"),
                html.Img(src="assets/pca1"+pcas2+".gif",style={'height':'130%', 'width':'95%'},className="d-flex align-self-center")]),
            dbc.Col([
                html.B("PC2"),
                html.Img(src="assets/pca2"+pcas2+".gif",style={'height':'130%', 'width':'95%'},className="d-flex align-self-center")]),
            dbc.Col([
                html.B("PC3"),
                html.Img(src="assets/pca3"+pcas2+".gif",style={'height':'130%', 'width':'95%'},className="d-flex align-self-center")]), 

    ])])])
    return plotpca

## Asparticos

In [22]:
#******************************** D I S T A N C I A   D E   A S P A R T I C O S*****************************************************   
@app.callback(
    Output("dplot", "children"),
    Input(component_id='dtplot', component_property='value'),         #Estructura deseada
    Input(component_id="img-format", component_property='data'),
)
def dist_cat(kind,f):  
    if kind=="hist":
        a = go.Figure(data=[go.Histogram(x=nat_dist_d[:,1], histnorm='probability',marker_color="royalblue",hovertext="Media: "+str(nat_dist_d[:,1].mean()))])
        a.update_layout(title="Distancia entre los asparticos cataliticos <br> Estructura Nativa",title_x=0.5,yaxis_range=[0,0.026],xaxis_range=[3.5,11])
        a.update_yaxes(title="Probability")
        a.update_xaxes(title="Distancia (Å)")
        e = go.Figure(data=[go.Histogram(x=mut_dist_d[:,1], histnorm='probability',marker_color="royalblue",hovertext="Media: "+str(mut_dist_d[:,1].mean()))])
        e.update_layout(title="Distancia entre los asparticos cataliticos <br> Estructura Nativa-L166P",title_x=0.5,yaxis_range=[0,0.026],xaxis_range=[3.5,11])
        e.update_yaxes(title="Probability")
        e.update_xaxes(title="Distancia (Å)")
        plot= html.Div([
            dbc.Row([
                dbc.Col([
                    dcc.Graph(figure=a,config=f)],width={"size":4,"offset":0}),
            dbc.Col([dcc.Graph(figure=e,config=f)],width={"size":4,"offset":0}),
            ])])
    if kind=="tiempo":
        a = go.Figure()
        a.add_traces(go.Scatter(x=nat_dist_d[:,0],y=nat_dist_d[:,1],marker_color="royalblue"))
        a.update_layout(title="Distancia entre los asparticos cataliticos <br> Estructura Nativa",title_x=0.5,yaxis_range=[3,11])
        a.update_xaxes(title="Tiempo (ns)")
        a.update_yaxes(title="Distancia (Å)")
        e = go.Figure()
        e.add_traces(go.Scatter(x=mut_dist_d[:,0],y=mut_dist_d[:,1],marker_color="royalblue"))
        e.update_layout(title="Distancia entre los asparticos cataliticos <br> Estructura Nativa-L166P",title_x=0.5,yaxis_range=[3,11])
        e.update_xaxes(title="Tiempo (ns)")
        e.update_yaxes(title="Distancia (Å)")
        plot= html.Div([
                dbc.Row([
                    dbc.Col([
                        dcc.Graph(figure=a,config=f)],width={"size":4,"offset":0}),

                dbc.Col([dcc.Graph(figure=e,config=f)],width={"size":4,"offset":0}),
                ])])
    return plot

## Lipidos

In [23]:
#********************************A N A L I S I S  D E  L I P I D O S*****************************************************   
@app.callback(
    Output("liplot", "children"),
    Input(component_id="lipy", component_property='value'),
    Input(component_id="img-format", component_property='data'),
)
def lipidos(ana,f):
    if ana =="order":
        order_nat  = np.genfromtxt("assets/order_density_nat.dat")
        order_gly  = np.genfromtxt("assets/order_density_gly.dat")
        order_mut  = np.genfromtxt("assets/order_density_mut.dat")
        order_cpx  = np.genfromtxt("assets/order_density_cpx.dat")
        order_sus  = np.genfromtxt("assets/order_density_sus.dat")
        n = go.Figure()
        n.add_traces(go.Scatter(x=[i for i in range(2,17)],y=np.mean(order_nat[:,1:6:2],axis=1),name="Palmitoil",line=dict(color="steelblue"),error_y=dict(type='data',array=np.mean(order_nat[:,2:7:2],axis=1),visible=True)))
        n.add_traces(go.Scatter(x=[i for i in range(2,19)],y=np.mean(order_nat[:,7::2],axis=1),name="Oleoil",line=dict(color="indianred"),error_y=dict(type='data',array=np.mean(order_nat[:,8::2],axis=1),visible=True)))
        n.update_layout(title="Sistema: Estructura Nativa <br> Parametro de Orden ",title_x=0.5,yaxis_range=[-0.05,0.25])
        n.update_xaxes(title="Número de carbono")
        n.update_yaxes(title="SCD")

        m = go.Figure()
        m.add_traces(go.Scatter(x=[i for i in range(2,17)],y=np.mean(order_mut[:,1:6:2],axis=1),name="Palmitoil",line=dict(color="steelblue"),error_y=dict(type='data',array=np.mean(order_mut[:,2:7:2],axis=1),visible=True)))
        m.add_traces(go.Scatter(x=[i for i in range(2,19)],y=np.mean(order_mut[:,7::2],axis=1),name="Oleoil",line=dict(color="indianred"),error_y=dict(type='data',array=np.mean(order_mut[:,8::2],axis=1),visible=True)))
        m.update_layout(title="Sistema: Estructura Nativa L166P <br> Parametro de Orden ",title_x=0.5,yaxis_range=[-0.05,0.25])
        m.update_xaxes(title="Número de carbono")
        m.update_yaxes(title="SCD")
    #########################################################################################################################################################################
    if ana =="mass":
        color = [None, "slategray","silver","darkgray","lightgrey","darkgray","lightgrey","royalblue","cornflowerblue","firebrick","tomato","black","gray", "purple","violet"]
        names =["#Z","PC", "sd(:PC)","OL","sd(:OL)","PA","sd(:PA)","WAT","sd(:WAT)","POPC","sdPOPC","Protein","sdProtein"]
        density_nat  = pd.read_csv("assets/nat_density.csv").values
        density_mut  = pd.read_csv("assets/mut_density.csv").values
        n = go.Figure()
        for i in range (1,density_nat.shape[1],2):
            n.add_traces(go.Scatter(x=density_nat[:,0],y=density_nat[:,i]+density_nat[:,i+1],showlegend=False,marker_color=color[i+1]))
            n.add_traces(go.Scatter(x=density_nat[:,0],y=density_nat[:,i]-density_nat[:,i+1],showlegend=False,marker_color=color[i+1],
                                      fill="tonexty"))
            n.add_traces(go.Scatter(x=density_nat[:,0],y=density_nat[:,i],name=names[i],
                                     marker_color=color[i]))
        n.update_layout(title="Sistema: Estructura Nativa <br> Perfil de densidad de membrana <br> Ancho membrana: "+anchomem(density_nat) +"Å",title_x=0.5)
        n.update_xaxes(title="Eje Z (Å)")
        n.update_yaxes(title="Densidad kg/m³")
        m = go.Figure()
        for i in range (1,density_mut.shape[1],2):
            m.add_traces(go.Scatter(x=density_mut[:,0],y=density_mut[:,i]+density_mut[:,i+1],showlegend=False,marker_color=color[i+1]))
            m.add_traces(go.Scatter(x=density_mut[:,0],y=density_mut[:,i]-density_mut[:,i+1],showlegend=False,marker_color=color[i+1],
                                      fill="tonexty"))
            m.add_traces(go.Scatter(x=density_mut[:,0],y=density_mut[:,i],name=names[i],
                                     marker_color=color[i]))
        m.update_layout(title="Sistema: Estructura Nativa L166P <br> Perfil de densidad de membrana <br> Ancho membrana: "+anchomem(density_mut) +"Å",title_x=0.5)
        m.update_xaxes(title="Eje Z (Å)")
        m.update_yaxes(title="Densidad kg/m³")
    ##############################################################################################################################################################################################################
    
    plot= html.Div([
    dbc.Row([
        dbc.Col([
            dcc.Graph(figure=n,config=f)],width={"size":4,"offset":0}),

    dbc.Col([dcc.Graph(figure=m,config=f)],width={"size":4,"offset":0})])])
    return (plot)

In [ ]:
server = app.server
# =============================================================================
if __name__ == '__main__':               #Dash se basa un poco en flask por eso esa sintaxis
    app.run_server(port = 1945) 
# =============================================================================
# if __name__ == '__main__':               #Dash se basa un poco en flask por eso esa sintaxis
#     app.run_server(debug=True) 

Dash is running on http://127.0.0.1:1945/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:1945
Press CTRL+C to quit
127.0.0.1 - - [08/Nov/2023 10:49:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2023 10:49:28] "GET /_dash-component-suites/dash/deps/react@16.v2_7_0m1669048543.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2023 10:49:28] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_7_0m1669048543.8.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2023 10:49:28] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_7_0m1669048543.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2023 10:49:28] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_7_0m1669048543.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2023 10:49:28] "GET /_dash-component-suites/dash_bootstrap_components/_components/dash_bootstrap_components.v1_3_1m1675042473.min.js HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2023 10:49:28] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_7_0m1669048543.min.js HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2023 10:49

# Fin